# Optimized Meal Planning

## Define the model

Data

$$
\begin{aligned}
T, \text{the set of discrete time periods} \\
I, \text{the set of ingredients that are available} \\
N, \text{the set of nutrients that are tracked} \\
R, \text{the set of known recipes} \\
F_{in}, \text{the amount of nutrient } n \text{ in ingredient } i,\ \forall i \in I,\ n \in N \\
S_{n}, \text{the amount of nutrient } n \text{ that is needed each week for a healthy diet} \\
C_{i}, \text{the cost of a single quantity of ingredient } i,\ \forall i \in I \\
V_{i}, \text{the volume of a single quantity of ingredient } i,\ \forall i \in I \\
Fr, \text{the max volume of the fridge} \\
Req_{ir}, \text{the amount of ingredient i needed for recipe} \ l \ \forall i \in I, r \in R \\
MAX, \text{the maximum number of recipes that can be made in a week}
\end{aligned}
$$

Only consider whether or not to buy an ingredient as an integer variable. The argument for this is that in real life, a recipe can be made with any amount of an ingredient but stores usually sell ingredients in preset sizes.

Decision variables:
- $x_{it}$, binary variable representing if ingredient i bought in week k, $\forall i \in I, t \in T$
- $y_{rt}$, the amount of recipe $l$ to make in week k, $\forall r \in R, t \in T$
- $z_{it}$, the amount of ingredient i to store in the fridge in week k, $\forall i \in I, t \in T$

Integer Program:
\begin{align}
\min \sum_{t \in T} \sum_{i \in I} x_{it} * C_i \\
\text{s.t. } \sum_{r \in R} \sum_{i \in I} y_{rt} * Req_{ir} * F_{in} \geq S_{n}, \forall t \in T, n \in N \\
\sum_{i \in I} z_{it} * V_{i} \leq Fr, \forall t \in T \\
z_{i0} = 0, \forall i \in I \\
x_{it}*V_i + z_{i,t-1} = \sum_{r \in R} y_{rt} * Req_{ir} + z_{it}, \forall t \in T, i \in I \\
\sum_{r \in R} y_{rt} \leq MAX, \forall t \in T \\
y_{rt}, z_{it} \geq 0, \forall i \in I, t \in T, x_{it} \text{ is binary}
\end{align}

- The first constraint ensures there are enough nutrients
- The second constraint ensures that the fridge is not overfilled
- The third constraint sets the intial amount of food to 0
- The fourth constraint ensures that the amount of each ingredient being consumed is balanced by the amount being bought
- The fifth constraint limits the amount of recipes made in a week (i.e. only 7 recipes worth of recipes being made)

In [12]:
# for now leave out multi-period planning and general form - too complex
using Random, NamedArrays

T_end = 10
T = 1:T_end
FR_CAP = 100

# Data 1:
I = [:rice, :chicken_nuggets, :steak]
N = [:calories, :protein]
R = [:steak_and_rice, :just_nuggets]
Req = NamedArray([5 0 3; 0 10 0], (R, I), ("Recipes", "Ingredients")) 
F = NamedArray([300 0; 200 5; 400 30], (I, N), ("Ingredients", "Nutrients"))
S = NamedArray([2000, 100], N, "Nutrient Requirements")
C = NamedArray([5 5 15; 5 5 15; 5 5 15; 5 5 15; 5 5 15;
    5 5 15; 5 5 15; 5 5 15; 5 5 15; 5 5 15;],
    (T, I), ("Week", "Ingredients"))
V = NamedArray([20, 20, 40], I, "Volume")

#Data 2:
# Random.seed!(300)
# I = ["rice", "chicken nuggets", "steak",
# "apple", "orange", "celery", "yogurt"]
# cost of item is roughly equivalent to string length
# C = [length(I)]
# for i in length(I)
#     C[i] = length(I[i]) + 2 * Random.randn()
# end

# make the model
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[T, I], Bin)
@variable(m, y[T, R] >= 0)
@variable(m, z[T, I] >= 0)

# Indexing Guide:
#
# t - time, i - ingredient, r - recipe, n - nutrient
# x[t, i], y[t, r], Req[r, i], F[i, n]

@constraint(m, nutr_satisfied[n in N, t in T],
    sum(y[t, r] * sum(Req[r, i] * F[i, n] for i in I) for r in R) >= S[n])

#@constraint(m, enough_ingr_for_recipes[i in I, t in T],
  #  x[t, i]*V[i] >= sum( Req[r, i]*y[t, r] for r in R )) 

@constraint(m, fridge_capacity[t in T], sum( x[t, i]*V[i] for i in I ) <= FR_CAP)

@constraint(m, first_ingredient_bal[i in I], x[1, i]*V[i] == sum( y[1,r] * Req[r,i] for r in R) + z[1, i])

@constraint(m, ingredient_bal[t in 2:T_end, i in I], x[t, i]*V[i] + z[t-1, i] == sum( y[t,r] * Req[r,i] for r in R) + z[t, i])

@objective(m, Min, sum( x[t, i] * C[t, i] for i in I, t in T ))

set_silent(m)
optimize!(m)

ingredient_values = NamedArray( [ (value(x[t,i])) for i in I, t in T ], (I, T), ("Ingredients", "Week"))
recipe_values = NamedArray( [ (value(y[t,r])) for r in R, t in T ], (R, T), ("Recipes", "Week"))
fridge_values = NamedArray( [ (value(z[t,i])) for i in I, t in T ], (I, T), ("Stored", "Week"))

println("What ingredients to buy in each week")
display(ingredient_values)
println()

println("Recipes cooked per week")
display(recipe_values)
println()

println("How much of each ingredient to store each week")
display(fridge_values)
println()

# println("Buy ingredients ", value.(x))
# println("Make recipes ", value.(y))
println("Total cost ", objective_value(m))

What ingredients to buy in each week


3×10 Named Matrix{Float64}
Ingredients ╲ Week │   1    2    3    4    5    6    7    8    9   10
───────────────────┼─────────────────────────────────────────────────
rice               │ 1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
chicken_nuggets    │ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
steak              │ 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


Recipes cooked per week


2×10 Named Matrix{Float64}
Recipes ╲ Week │       1        2        3  …        8        9       10
───────────────┼────────────────────────────────────────────────────────
steak_and_rice │ 1.11111  1.11111  1.11111  …  1.11111  1.11111  1.11111
just_nuggets   │     0.0      0.0      0.0  …      0.0      0.0      0.0


How much of each ingredient to store each week


3×10 Named Matrix{Float64}
  Stored ╲ Week │       1        2        3  …        8        9       10
────────────────┼────────────────────────────────────────────────────────
rice            │ 14.4444  8.88889  23.3333  …  15.5556     10.0  4.44444
chicken_nuggets │     0.0      0.0      0.0         0.0      0.0      0.0
steak           │ 36.6667  33.3333     30.0  …  13.3333     10.0  6.66667


Total cost 30.0


In [9]:
#using DataFrames, CSV
#df = CSV.read("stigler.csv",DataFrame,delim=',')